In [1]:
import psycopg2

In [2]:
from printsql import printsql

In [3]:
conn=psycopg2.connect(database="postgres")

In [4]:
cur=conn.cursor()

In [5]:
#"\du" 对应的sql语句
du = '''
SELECT u.usename AS "User name",
  u.usesysid AS "User ID",
  CASE WHEN u.usesuper AND u.usecreatedb THEN CAST('superuser, create
database' AS pg_catalog.text)
       WHEN u.usesuper THEN CAST('superuser' AS pg_catalog.text)
       WHEN u.usecreatedb THEN CAST('create database' AS
pg_catalog.text)
       ELSE CAST('' AS pg_catalog.text)
  END AS "Attributes"
FROM pg_catalog.pg_user u
ORDER BY 1;
'''

In [6]:
cur.execute(du)
printsql (cur, 10)

User name User ID   Attributes
dbuser1   16390               
postgres  10        superuser, create
database
yichang   32768     superuser 


In [27]:
cur.execute("Create User dbuser2 WITH PASSWORD '123456';")

In [28]:
# 显示已经添加的用户；
# conn.commit()之后才生效；
cur.execute(du)
printsql (cur)

User name User ID   Attributes
dbuser1   16390               
dbuser2   32771               
postgres  10        superuser, create
database
yichang   32768     superuser 


In [20]:
#Some PostgreSQL command such as CREATE DATABASE or VACUUM can’t run into a transaction: to run such command use:

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [21]:
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [22]:
cur = conn.cursor()

In [24]:
cur.execute("Create DATABASE testdb2;")

In [29]:
conn.commit()

In [30]:
cur.execute("GRANT ALL PRIVILEGES ON DATABASE testdb2 to dbuser2;")

In [31]:
cur.execute(du)
printsql (cur)

User name User ID   Attributes
dbuser1   16390               
dbuser2   32771               
postgres  10        superuser, create
database
yichang   32768     superuser 


In [36]:
#\l 替代命令
l = '''
SELECT datname FROM pg_database
WHERE datistemplate = false;
'''

In [37]:
cur.execute(l)
printsql (cur)

datname   
postgres  
testdb    
testdb2   


In [38]:
conn.commit()
cur.close()
conn.close()

### 采用子线程执行psql命令
cur.execute("\du") 会失败，通过子进程会成功；  


In [44]:
conn=psycopg2.connect(database="postgres",user="postgres",password="123456",host="localhost",port="5432")
cur=conn.cursor()

In [48]:
# 子进程没有当前的联接
du = "psql -d testdb -c '\du'"

In [49]:
import subprocess
import sys 

In [50]:
res = subprocess.run(du, stdout = subprocess.PIPE)
print(res.stdout.decode(sys.stdout.encoding))

FileNotFoundError: [Errno 2] No such file or directory: "psql -d testdb -c '\\du'": "psql -d testdb -c '\\du'"